# Infusion Recipe Suggestions

This notebook uses the ingredient network to suggest fusion recipes by combining ingredients from different cuisines.

## Overview

The infusion suggestion process:
1. Load the ingredient network
2. Identify ingredient sets for different cuisines
3. Find "bridge" ingredients that connect cuisines
4. Generate fusion recipe suggestions
5. Rank suggestions by network connectivity and balance


In [ ]:
# Setup
import sys
from pathlib import Path
import networkx as nx
import pandas as pd

pipeline_root = Path.cwd().parent
if str(pipeline_root) not in sys.path:
    sys.path.insert(0, str(pipeline_root))

from network.graph import IngredientGraph
from inference.infusion_suggestor import InfusionSuggestor


## Step 1: Load Network and Data


In [ ]:
# Load network
graph_path = Path("./data/ingredient_network.graphml")
if graph_path.exists():
    graph = nx.read_graphml(graph_path)
    ing_graph = IngredientGraph(graph)
    print(f"Loaded network: {ing_graph.num_nodes:,} nodes, {ing_graph.num_edges:,} edges")
else:
    print(f"Network not found: {graph_path}")
    print("Please run 01_build_network.ipynb first!")

# Load recipe data to get cuisine ingredient sets
data_path = Path("../preprocess_pipeline/data/encoded_combined_datasets_with_cuisine_encoded.parquet")
df = pd.read_parquet(data_path)
print(f"\nLoaded {len(df):,} recipes")


## Step 2: Prepare Cuisine Ingredient Sets

Extract ingredient sets for different cuisines.


In [ ]:
# Group ingredients by cuisine
cuisine_ingredients = {}
for _, row in df.iterrows():
    cuisines = row.get('cuisine_encoded', [])
    ingredients = row.get('encoded_ingredients', [])
    
    # Handle list formats
    if isinstance(cuisines, (list, tuple)):
        cuisine_list = [int(c) for c in cuisines if c and c != 0]
    else:
        cuisine_list = []
    
    if isinstance(ingredients, (list, tuple)):
        ing_list = [int(i) for i in ingredients if i and i != 0]
    else:
        ing_list = []
    
    # Add ingredients to each cuisine
    for cuisine_id in cuisine_list:
        if cuisine_id not in cuisine_ingredients:
            cuisine_ingredients[cuisine_id] = set()
        cuisine_ingredients[cuisine_id].update(ing_list)

print(f"Found {len(cuisine_ingredients)} cuisines")
for cuisine_id, ing_set in list(cuisine_ingredients.items())[:5]:
    print(f"  Cuisine {cuisine_id}: {len(ing_set)} unique ingredients")


## Step 3: Generate Infusion Suggestions

Use the network to suggest fusion recipes between two cuisines.


In [ ]:
# Initialize suggestor
suggestor = InfusionSuggestor(ing_graph)

# Example: Suggest fusion between first two cuisines
if len(cuisine_ingredients) >= 2:
    cuisine_ids = list(cuisine_ingredients.keys())[:2]
    cuisine1_id, cuisine2_id = cuisine_ids[0], cuisine_ids[1]
    
    cuisine1_ings = cuisine_ingredients[cuisine1_id]
    cuisine2_ings = cuisine_ingredients[cuisine2_id]
    
    print(f"Generating fusion suggestions between Cuisine {cuisine1_id} and Cuisine {cuisine2_id}")
    print(f"  Cuisine {cuisine1_id}: {len(cuisine1_ings)} ingredients")
    print(f"  Cuisine {cuisine2_id}: {len(cuisine2_ings)} ingredients")
    
    suggestions = suggestor.suggest_fusion_recipes(
        cuisine1_ingredients=cuisine1_ings,
        cuisine2_ingredients=cuisine2_ings,
        max_suggestions=10,
    )
    
    print(f"\nGenerated {len(suggestions)} fusion suggestions:")
    for i, suggestion in enumerate(suggestions, 1):
        print(f"\n  Suggestion {i}:")
        print(f"    Score: {suggestion['score']:.4f}")
        print(f"    Ingredients: {len(suggestion['ingredients'])}")
        print(f"    Cuisine 1 ratio: {suggestion['cuisine1_ratio']:.2%}")
        print(f"    Cuisine 2 ratio: {suggestion['cuisine2_ratio']:.2%}")
        print(f"    Sample ingredients: {suggestion['ingredients'][:5]}")
